## Carga librerias necesarias


In [36]:
import pandas as pd 
import numpy as np
import os
from pandas import concat

## Carga del los dataset correspondientes al año 2022
Se cargan los csv de cada mes en una lista

In [37]:
file_names = os.listdir('../data/Anio22/')
file_names

['abr_mo22.csv',
 'ago_mo22.csv',
 'dic_mo22.csv',
 'ene_mo22.csv',
 'feb_mo22.csv',
 'jul_mo22.csv',
 'jun_mo22.csv',
 'mar_mo22.csv',
 'may_mo22.csv',
 'nov_mo22.csv',
 'oct_mo22.csv',
 'sep_mo22.csv']

## Crear único dataset con los los datos del año 2022
Recorrer la lista files_names para crear un único dataset


In [38]:
df_aire = pd.DataFrame()
for file_name in file_names:
    df_current = pd.read_csv('../data/Anio22/' + file_name, delimiter=';')
    print(file_name, df_current.shape)
    df_aire = pd.concat([df_aire, df_current], ignore_index=True)
    

abr_mo22.csv (3951, 56)
ago_mo22.csv (3960, 56)
dic_mo22.csv (4092, 56)
ene_mo22.csv (3929, 56)
feb_mo22.csv (3556, 56)
jul_mo22.csv (3968, 56)
jun_mo22.csv (3872, 56)
mar_mo22.csv (3946, 56)
may_mo22.csv (4059, 56)
nov_mo22.csv (3960, 56)
oct_mo22.csv (4092, 56)
sep_mo22.csv (3838, 56)


## Filtrar los datos con el componente NO2

In [39]:
df_aire = df_aire[(df_aire['MAGNITUD'] == 8)]
df_aire.head()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
90,28,79,4,8,28079004_8_8,2022,4,1,16.0,V,...,26.0,V,33.0,V,35.0,V,24.0,V,20.0,V
91,28,79,4,8,28079004_8_8,2022,4,2,13.0,V,...,18.0,V,23.0,V,24.0,V,18.0,V,15.0,V
92,28,79,4,8,28079004_8_8,2022,4,3,16.0,V,...,13.0,V,15.0,V,19.0,V,12.0,V,9.0,V
93,28,79,4,8,28079004_8_8,2022,4,4,8.0,V,...,28.0,V,28.0,V,29.0,V,32.0,V,19.0,V
94,28,79,4,8,28079004_8_8,2022,4,5,16.0,V,...,22.0,V,17.0,V,19.0,V,20.0,V,16.0,V


## Crear una nueva columna fecha 

In [40]:
df_aire['FECHA'] = pd.to_datetime(df_aire[['ANO','MES','DIA']].rename(columns = {'ANO': 'YEAR', 'MES': 'MONTH', 'DIA': 'DAY'}))

## Extraer las datos de las medidas tomadas

In [41]:
data_no2_columns = [ 'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'H24'] # feature
veri_columns = [ 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24']

hora_column = np.arange(1,25) # feature label

## Crear un dataset auxiliar con los datos extraidos  y las nuevas columnas creadas

In [42]:


data_no2 = pd.DataFrame(columns=['HORA', 'DATO', 'VERIFICADO'])
aux = pd.DataFrame(columns=['ANO', 'MES', 'DIA', 'HORA', 'DATO', 'VERIFICADO', 'ESTACION']) # aux de próxima fila a ser añadida
for  (h,d,v) in zip(hora_column, data_no2_columns, veri_columns): 
     aux['DATO'] = df_aire [ data_no2_columns[h-1] ] 
     aux['ICA_NO2'] = (df_aire[data_no2_columns[h-1] ]/50)*100
     aux['VERIFICADO'] = df_aire [ veri_columns[h-1] ] 
     aux['HORA'] = h-1
     aux[['ANO', 'MES', 'DIA']] = df_aire[['ANO','MES','DIA']]
     aux['ESTACION'] = df_aire ['ESTACION']
     data_no2 = pd.concat([data_no2, aux], ignore_index=True)
data_no2 = data_no2.astype({'HORA': 'int32'}, copy = False)

data_no2.head()



,HORA,DATO,VERIFICADO,ANO,MES,DIA,ESTACION,ICA_NO2
0,0,16.0,V,2022.0,4.0,1.0,4.0,32.0
1,0,13.0,V,2022.0,4.0,2.0,4.0,26.0
2,0,16.0,V,2022.0,4.0,3.0,4.0,32.0
3,0,8.0,V,2022.0,4.0,4.0,4.0,16.0
4,0,16.0,V,2022.0,4.0,5.0,4.0,32.0


## Añadir la columna fecha en formato datetime al nuevo dataset 

In [43]:
data_no2['FECHA'] = pd.to_datetime(data_no2[['ANO','MES','DIA','HORA']].rename(columns = {'ANO': 'year', 'MES': 'month','DIA': 'day', 'HORA': 'hour'}))
data_no2.head()

,HORA,DATO,VERIFICADO,ANO,MES,DIA,ESTACION,ICA_NO2,FECHA
0,0,16.0,V,2022.0,4.0,1.0,4.0,32.0,2022-04-01
1,0,13.0,V,2022.0,4.0,2.0,4.0,26.0,2022-04-02
2,0,16.0,V,2022.0,4.0,3.0,4.0,32.0,2022-04-03
3,0,8.0,V,2022.0,4.0,4.0,4.0,16.0,2022-04-04
4,0,16.0,V,2022.0,4.0,5.0,4.0,32.0,2022-04-05


## Multi-index data frame con estacion y fecha :O

In [44]:
data_no2.set_index(['ESTACION', 'FECHA'], inplace=True)
data_no2.head()

HORA  DATO VERIFICADO     ANO  MES  DIA  ICA_NO2
ESTACION FECHA                                                       
4.0      2022-04-01     0  16.0          V  2022.0  4.0  1.0     32.0
         2022-04-02     0  13.0          V  2022.0  4.0  2.0     26.0
         2022-04-03     0  16.0          V  2022.0  4.0  3.0     32.0
         2022-04-04     0   8.0          V  2022.0  4.0  4.0     16.0
         2022-04-05     0  16.0          V  2022.0  4.0  5.0     32.0

## Ordenar por estación y fecha 

In [45]:
data_no2 = data_no2.sort_values(['ESTACION', 'FECHA'])
data_no2.head()



HORA  DATO VERIFICADO     ANO  MES   DIA  \
ESTACION FECHA                                                           
4.0      2022-03-25 00:00:00     0   0.0          N  2022.0  3.0  25.0   
         2022-03-25 01:00:00     1   0.0          N  2022.0  3.0  25.0   
         2022-03-25 02:00:00     2   0.0          N  2022.0  3.0  25.0   
         2022-03-25 03:00:00     3   0.0          N  2022.0  3.0  25.0   
         2022-03-25 04:00:00     4   0.0          N  2022.0  3.0  25.0   

                              ICA_NO2  
ESTACION FECHA                         
4.0      2022-03-25 00:00:00      0.0  
         2022-03-25 01:00:00      0.0  
         2022-03-25 02:00:00      0.0  
         2022-03-25 03:00:00      0.0  
         2022-03-25 04:00:00      0.0

## Rellenar con NaN las filas perdidas 
 Para cada estación, rellenamos con NaN si hay alguna fila perdida y guardamos el resultado en un nuevo dataframe que ya debería tener todas las filas


In [46]:
estaciones = data_no2.index.get_level_values('ESTACION').unique()
datetime_index = pd.date_range(start='1/1/2022 00:00:00', end='12/31/2022 23:00:00', freq='H', name='FECHA')
data_no2_nuevo = pd.DataFrame()
for estacion in estaciones:
    #data_no2_estacion = data_no2[data_no2['estacion'] == estacion]
    data_no2_estacion = data_no2.loc[estacion]
    print(estacion, len(data_no2_estacion))
    data_no2_estacion = data_no2_estacion.reindex(datetime_index)
    data_no2_estacion.reset_index(inplace=True)
    print(estacion, len(data_no2_estacion))
    data_no2_estacion['ESTACION'] = estacion
    data_no2_nuevo = pd.concat([data_no2_nuevo, data_no2_estacion],ignore_index=True)
    
print(len(data_no2_nuevo))

4.0 6720
4.0 8760
8.0 8760
8.0 8760
11.0 8760
11.0 8760
16.0 6336
16.0 8760
17.0 8760
17.0 8760
18.0 8760
18.0 8760
24.0 8736
24.0 8760
27.0 8760
27.0 8760
35.0 8760
35.0 8760
36.0 8520
36.0 8760
38.0 8760
38.0 8760
39.0 8760
39.0 8760
40.0 8760
40.0 8760
47.0 8712
47.0 8760
48.0 8760
48.0 8760
49.0 8760
49.0 8760
50.0 8760
50.0 8760
54.0 8760
54.0 8760
55.0 8760
55.0 8760
56.0 8760
56.0 8760
57.0 8760
57.0 8760
58.0 8760
58.0 8760
59.0 8760
59.0 8760
60.0 8760
60.0 8760
210240


In [47]:
# Multi-index data_no2 frame con estacion y fecha :O
data_no2_nuevo.set_index(['ESTACION', 'FECHA'], inplace=True)

##  Actualizar  data con el nuevo df:

In [48]:
data_no2 = data_no2_nuevo
data_no2.info()
data_no2

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 210240 entries, (4.0, Timestamp('2022-01-01 00:00:00')) to (60.0, Timestamp('2022-12-31 23:00:00'))
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   HORA        205464 non-null  float64
 1   DATO        205464 non-null  float64
 2   VERIFICADO  205464 non-null  object 
 3   ANO         205464 non-null  float64
 4   MES         205464 non-null  float64
 5   DIA         205464 non-null  float64
 6   ICA_NO2     205464 non-null  float64
dtypes: float64(6), object(1)
memory usage: 12.1+ MB


HORA  DATO VERIFICADO     ANO   MES   DIA  \
ESTACION FECHA                                                            
4.0      2022-01-01 00:00:00   NaN   NaN        NaN     NaN   NaN   NaN   
         2022-01-01 01:00:00   NaN   NaN        NaN     NaN   NaN   NaN   
         2022-01-01 02:00:00   NaN   NaN        NaN     NaN   NaN   NaN   
         2022-01-01 03:00:00   NaN   NaN        NaN     NaN   NaN   NaN   
         2022-01-01 04:00:00   NaN   NaN        NaN     NaN   NaN   NaN   
...                            ...   ...        ...     ...   ...   ...   
60.0     2022-12-31 19:00:00  19.0  60.0          V  2022.0  12.0  31.0   
         2022-12-31 20:00:00  20.0  68.0          V  2022.0  12.0  31.0   
         2022-12-31 21:00:00  21.0  67.0          V  2022.0  12.0  31.0   
         2022-12-31 22:00:00  22.0  58.0          V  2022.0  12.0  31.0   
         2022-12-31 23:00:00  23.0  49.0          V  2022.0  12.0  31.0   

                              ICA_NO2  
ESTACION FECHA                         
4.0      2022-01-01 00:00:00      NaN  
         2022-01-01 01:00:00      NaN  
         2022-01-01 02:00:00      NaN  
         2022-01-01 03:00:00      NaN  
         2022-01-01 04:00:00      NaN  
...                               ...  
60.0     2022-12-31 19:00:00    120.0  
         2022-12-31 20:00:00    136.0  
         2022-12-31 21:00:00    134.0  
         2022-12-31 22:00:00    116.0  
         2022-12-31 23:00:00     98.0  

[210240 rows x 7 columns]

In [49]:
# Mostramos los datos que se habían perdidos
missing_data_no2 = data_no2[data_no2['DATO'].isna()]
print(f'Cantidad de datos no existentes (perdidos): {len(missing_data_no2)}')
missing_data_no2

Cantidad de datos no existentes (perdidos): 4776


HORA  DATO VERIFICADO  ANO  MES  DIA  ICA_NO2
ESTACION FECHA                                                             
4.0      2022-01-01 00:00:00   NaN   NaN        NaN  NaN  NaN  NaN      NaN
         2022-01-01 01:00:00   NaN   NaN        NaN  NaN  NaN  NaN      NaN
         2022-01-01 02:00:00   NaN   NaN        NaN  NaN  NaN  NaN      NaN
         2022-01-01 03:00:00   NaN   NaN        NaN  NaN  NaN  NaN      NaN
         2022-01-01 04:00:00   NaN   NaN        NaN  NaN  NaN  NaN      NaN
...                            ...   ...        ...  ...  ...  ...      ...
47.0     2022-03-13 19:00:00   NaN   NaN        NaN  NaN  NaN  NaN      NaN
         2022-03-13 20:00:00   NaN   NaN        NaN  NaN  NaN  NaN      NaN
         2022-03-13 21:00:00   NaN   NaN        NaN  NaN  NaN  NaN      NaN
         2022-03-13 22:00:00   NaN   NaN        NaN  NaN  NaN  NaN      NaN
         2022-03-13 23:00:00   NaN   NaN        NaN  NaN  NaN  NaN      NaN

[4776 rows x 7 columns]

In [50]:
# Rellenamos las columnas conciernientes a la fecha que faltan en esos datos perdidos excepto la columna 'dato' que ya trataremos más adelante
dt = missing_data_no2.index.get_level_values('FECHA')
data_no2.loc[data_no2['DATO'].isna(),'ANO'] = dt.year
data_no2.loc[data_no2['DATO'].isna(),'MES'] = dt.month
data_no2.loc[data_no2['DATO'].isna(),'DIA'] = dt.day
data_no2.loc[data_no2['DATO'].isna(),'HORA'] = dt.hour
data_no2 = data_no2.astype({'HORA': 'int32'}, copy = False)

#data_no2.loc[data_no2['dato'].isna(),['ano', 'mes', 'dia', 'hora']] = [dt.year, dt.month, dt.day, dt.hour]
data_no2[data_no2['DATO'].isna()]

HORA  DATO VERIFICADO     ANO  MES   DIA  \
ESTACION FECHA                                                           
4.0      2022-01-01 00:00:00     0   NaN        NaN  2022.0  1.0   1.0   
         2022-01-01 01:00:00     1   NaN        NaN  2022.0  1.0   1.0   
         2022-01-01 02:00:00     2   NaN        NaN  2022.0  1.0   1.0   
         2022-01-01 03:00:00     3   NaN        NaN  2022.0  1.0   1.0   
         2022-01-01 04:00:00     4   NaN        NaN  2022.0  1.0   1.0   
...                            ...   ...        ...     ...  ...   ...   
47.0     2022-03-13 19:00:00    19   NaN        NaN  2022.0  3.0  13.0   
         2022-03-13 20:00:00    20   NaN        NaN  2022.0  3.0  13.0   
         2022-03-13 21:00:00    21   NaN        NaN  2022.0  3.0  13.0   
         2022-03-13 22:00:00    22   NaN        NaN  2022.0  3.0  13.0   
         2022-03-13 23:00:00    23   NaN        NaN  2022.0  3.0  13.0   

                              ICA_NO2  
ESTACION FECHA                         
4.0      2022-01-01 00:00:00      NaN  
         2022-01-01 01:00:00      NaN  
         2022-01-01 02:00:00      NaN  
         2022-01-01 03:00:00      NaN  
         2022-01-01 04:00:00      NaN  
...                               ...  
47.0     2022-03-13 19:00:00      NaN  
         2022-03-13 20:00:00      NaN  
         2022-03-13 21:00:00      NaN  
         2022-03-13 22:00:00      NaN  
         2022-03-13 23:00:00      NaN  

[4776 rows x 7 columns]

In [51]:


# 6. Por último, marcamos los datos no verificados como NaN
no_verificados = data_no2.loc[data_no2['VERIFICADO'] != 'V']
print(f'Cantidad de datos existentes pero no válidos:{len(no_verificados)}')
data_no2.loc[data_no2['VERIFICADO'] != 'V', ['DATO']] = np.nan
data_no2.loc[data_no2['VERIFICADO'] != 'V']



Cantidad de datos existentes pero no válidos:6642


HORA  DATO VERIFICADO     ANO   MES   DIA  \
ESTACION FECHA                                                            
4.0      2022-01-01 00:00:00     0   NaN        NaN  2022.0   1.0   1.0   
         2022-01-01 01:00:00     1   NaN        NaN  2022.0   1.0   1.0   
         2022-01-01 02:00:00     2   NaN        NaN  2022.0   1.0   1.0   
         2022-01-01 03:00:00     3   NaN        NaN  2022.0   1.0   1.0   
         2022-01-01 04:00:00     4   NaN        NaN  2022.0   1.0   1.0   
...                            ...   ...        ...     ...   ...   ...   
60.0     2022-11-21 12:00:00    12   NaN          N  2022.0  11.0  21.0   
         2022-11-30 09:00:00     9   NaN          N  2022.0  11.0  30.0   
         2022-11-30 10:00:00    10   NaN          N  2022.0  11.0  30.0   
         2022-12-21 11:00:00    11   NaN          N  2022.0  12.0  21.0   
         2022-12-21 12:00:00    12   NaN          N  2022.0  12.0  21.0   

                              ICA_NO2  
ESTACION FECHA                         
4.0      2022-01-01 00:00:00      NaN  
         2022-01-01 01:00:00      NaN  
         2022-01-01 02:00:00      NaN  
         2022-01-01 03:00:00      NaN  
         2022-01-01 04:00:00      NaN  
...                               ...  
60.0     2022-11-21 12:00:00      8.0  
         2022-11-30 09:00:00     58.0  
         2022-11-30 10:00:00    104.0  
         2022-12-21 11:00:00      6.0  
         2022-12-21 12:00:00     20.0  

[6642 rows x 7 columns]

In [52]:
# borramos columna verificado
del data_no2['VERIFICADO']
data_no2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 210240 entries, (4.0, Timestamp('2022-01-01 00:00:00')) to (60.0, Timestamp('2022-12-31 23:00:00'))
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   HORA     210240 non-null  int32  
 1   DATO     203598 non-null  float64
 2   ANO      210240 non-null  float64
 3   MES      210240 non-null  float64
 4   DIA      210240 non-null  float64
 5   ICA_NO2  205464 non-null  float64
dtypes: float64(5), int32(1)
memory usage: 9.7 MB


In [53]:


# Primero de todo, deshacemos el multíndice
data_no2.reset_index(inplace=True) # Salvar los datos sin indices de estacion y fecha



In [54]:


# 7. Añadimos columna para el día de la semana
#dates = data_no2.index.get_level_values('fecha').to_pydatetime() # con multiindex
dates = data_no2['FECHA'].dt.to_pydatetime() # con column y sin multiindex
days_of_week = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
data_no2['DIA_SEMANA'] = np.array([days_of_week[d.weekday()] for d in dates])



In [55]:
data_no2 = data_no2.rename(columns={'DATO': 'NO2'}, copy = False)

In [56]:


data_no2.info()
data_no2.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210240 entries, 0 to 210239
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   ESTACION    210240 non-null  float64       
 1   FECHA       210240 non-null  datetime64[ns]
 2   HORA        210240 non-null  int32         
 3   NO2         203598 non-null  float64       
 4   ANO         210240 non-null  float64       
 5   MES         210240 non-null  float64       
 6   DIA         210240 non-null  float64       
 7   ICA_NO2     205464 non-null  float64       
 8   DIA_SEMANA  210240 non-null  object        
dtypes: datetime64[ns](1), float64(6), int32(1), object(1)
memory usage: 13.6+ MB


,ESTACION,FECHA,HORA,NO2,ANO,MES,DIA,ICA_NO2,DIA_SEMANA
0,4.0,2022-01-01 00:00:00,0,NaN,2022.0,1.0,1.0,NaN,Sabado
1,4.0,2022-01-01 01:00:00,1,NaN,2022.0,1.0,1.0,NaN,Sabado
2,4.0,2022-01-01 02:00:00,2,NaN,2022.0,1.0,1.0,NaN,Sabado
3,4.0,2022-01-01 03:00:00,3,NaN,2022.0,1.0,1.0,NaN,Sabado
4,4.0,2022-01-01 04:00:00,4,NaN,2022.0,1.0,1.0,NaN,Sabado


In [57]:
data_no2.tail()

,ESTACION,FECHA,HORA,NO2,ANO,MES,DIA,ICA_NO2,DIA_SEMANA
210235,60.0,2022-12-31 19:00:00,19,60.0,2022.0,12.0,31.0,120.0,Sabado
210236,60.0,2022-12-31 20:00:00,20,68.0,2022.0,12.0,31.0,136.0,Sabado
210237,60.0,2022-12-31 21:00:00,21,67.0,2022.0,12.0,31.0,134.0,Sabado
210238,60.0,2022-12-31 22:00:00,22,58.0,2022.0,12.0,31.0,116.0,Sabado
210239,60.0,2022-12-31 23:00:00,23,49.0,2022.0,12.0,31.0,98.0,Sabado


In [58]:
# Serie con solamente los datos de contaminación
datos = data_no2['NO2']
datos

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
210235    60.0
210236    68.0
210237    67.0
210238    58.0
210239    49.0
Name: NO2, Length: 210240, dtype: float64

In [59]:
# datos.to_csv('data_no2_hist.csv') # Salvado serie solo datos históricos
# data_no2.to_csv('data_n02_fecha.csv') # Salvado con los datos de las fechas